In [1]:
from functools import partial
from pathlib import Path

from killscreen.utilities import roundstring

from s3_fuse.bench_utils import (
    summarize_stat,
    check_existing_benchmarks,
    dump_bandwidth_allowance_metrics,
    dump_throwaway_results
)
from s3_fuse.handlers import interpret_benchmark_instructions, execute_test_case, process_bench_stats
from s3_fuse.mount_s3 import mount_bucket

simple execution environment for dataset benchmarking. executes benchmarks defined in the benchmark_settings module.

important notes:
* the `fsspec`-based "s3" and "s3_section" loaders will only function on the barentsen/cloud-support astropy branch.
* other loaders will only function if `goofys` is executable from the user's path.
* attempts to access private buckets, via either `goofys` or `fsspec`, will use credentials stored in ~/.aws/credentials.
* bandwidth throttling will only work if `wondershaper` is executable from the user's path, the user is a sudoer, and the user has no `sudo` password (like "ubuntu" in stock AWS EC2 Ubuntu images).
* this currently assumes that the network interface of interest is named "ens5"


In [ ]:
SETTINGS = {
    # directory goofys will use as a mount point for buckets
    "mountpoint": "/mnt/s3/",
    # max files -- will use the top n_files elements of the TEST_FILES attribute of
    # each benchmark module (unless less than n_files are given in TEST_FILES!)
    "n_files": 25,
    # do we actually want to look at the cuts? probably not in big bulk benchmarks,
    # but can be useful for diagnostics or if you want a screensaver or whatever.
    "return_cuts": False,
    # rng seed for consistent execution w/out having to explicitly define a very long
    # list of rectangles
    "seed": 123456,
    # these values are given in kilobits per second. None means unthrottled.
    # if you don't pass this key, there won't be any throttling in any test.
    "bandwidth": (None, 100 * 1000)
}
# these correspond to the names of submodules of the benchmark_settings module
BENCHMARK_NAMES = (
    "hst", "panstarrs", "galex_rice", "jwst_crf", "tesscut", "galex_gzip", "hst_big", 
)
# if there is an existing result file corresponding to a particular test case, shall
# we run it again (wiincrementing suffixes attached to outputs?)
DUPLICATE_BENCHMARKS = False
# where shall we write benchmark results?
METRIC_DIRECTORY = "bench_results"
# how many throwaway tests should we run every time we switch the specific set of cuts we're using?
# this is intended to juice S3 serverside caching so that earlier-in-order
# cases aren't 'penalized' without us having to either make a separate copy of all S3 objects
# for every test case, select random objects from a larger corpus, or wait a long time (60s - 15m, ish)
# between each test case.
# this is spooky, because it's a black box -- we have no way to seriously interrogate
# how this works. It can affect run times by a factor of 2-3 in many cases, though.
N_THROWAWAYS = 3
# "benchmarks" contains instructions for each test case
benchmarks = {
    name: interpret_benchmark_instructions(name, SETTINGS)
    for name in BENCHMARK_NAMES
}
# partially-evaluated S3 mount function, for convenience
remount = partial(
    mount_bucket, "goofys", SETTINGS['mountpoint'], remount=True
)

In [ ]:
"""
execute all benchmarks in a loop and save results. you can mess with this cell
to pretty straightforwardly look at only certain categories of cases,
suppress results, etc.
"""
logs = {}
for benchmark_name, benchmark in benchmarks.items():
    previous = {}
    for case in benchmark:
        skip, suffix = check_existing_benchmarks(case['title'], DUPLICATE_BENCHMARKS, METRIC_DIRECTORY)
        if skip is True:
            continue
        cuts, stat, log, throwaways = execute_test_case(case, previous, remount, N_THROWAWAYS)
        print(roundstring(summarize_stat(stat)) + "\n")
        logs[f"{case['title']}"] = log
        process_bench_stats(log, case, benchmark_name).to_csv(
            Path(METRIC_DIRECTORY, f"{case['title']}_{suffix}.csv"), index=None
        )
        dump_bandwidth_allowance_metrics(case['title'], suffix, METRIC_DIRECTORY)
        dump_throwaway_results(throwaways, case['title'], suffix, METRIC_DIRECTORY)
        previous = case
